## Assignment 01. Simple text processing.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from IPython import display

### Toxic or not
Your main goal in this assignment is to classify, whether the comments are toxic or not. And practice with both classical approaches and PyTorch in the process.

*Credits: This homework is inspired by YSDA NLP_course.*

In [4]:
# In colab run this cell
! wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/advanced_f20/homeworks_advanced/assignment1_01_Word_Vectors/utils.py -nc
! wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/advanced_f20/homeworks_advanced/assignment1_01_Word_Vectors/features.py -nc

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [5]:
try:
    data = pd.read_csv('../../datasets/comments_small_dataset/comments.tsv', sep='\t')
except FileNotFoundError:
    ! wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/advanced_f20/datasets/comments_small_dataset/comments.tsv -nc
    data = pd.read_csv("comments.tsv", sep='\t')

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


FileNotFoundError: [Errno 2] No such file or directory: 'comments.tsv'

In [4]:
texts = data['comment_text'].values
target = data['should_ban'].values
data[50::200]

,should_ban,comment_text
50,0,"""Those who're in advantageous positions are th..."
250,1,Fartsalot56 says f**k you motherclucker!!
450,1,"Are you a fool? \n\nI am sorry, but you seem t..."
650,1,I AM NOT A VANDAL!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
850,0,Citing sources\n\nCheck out the Wikipedia:Citi...


In [5]:
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(texts, target, test_size=0.5, random_state=42)

In [7]:
texts_test

array(['You need a life \n\nREALLY BAD.......................BITCH!',
       'User_talk:TShilo12#Witkacy_WP:POINT  Also, Jayjg got',
       'Your blocks do not deter me \nI may be blocked but you are still an asshole. And coward.  93.138.126.185',
       '"\n\n More Answers \n\n \nPATTY SAID\n\n<<  It clearly shows in the history that others furthered the breed by taking them to America and England. Thus the same with the Shiloh Shepherd. Some facts: While the ISSR was founded in 1991 as the first Shiloh registry, it was only 6 short years later that the first non ISSR registry and club was formed. Thus the ISSR was only the sole Shiloh Registry for 6 years while there have been non ISSR registries for 8.  >>\n\nTINA\'S  REPLY\n\nROTF ... Talk about ""smoke & mirrors"" ... WOW!!  Now let\'s look at some *varifiable* FACTS!\n\n1. The ISSR was formed to CONTINUE the data collection that I started in 1974!!  When it was set up I had documented data on MY breeding stock going all the way b

__Note:__ it is generally a good idea to split data into train/test before anything is done to them.

It guards you against possible data leakage in the preprocessing stage. For example, should you decide to select words present in obscene tweets as features, you should only count those words over the training set. Otherwise your algoritm can cheat evaluation.

### Preprocessing and tokenization

Comments contain raw text with punctuation, upper/lowercase letters and even newline symbols.

To simplify all further steps, we'll split text into space-separated tokens using one of nltk tokenizers.

Generally, library `nltk` [link](https://www.nltk.org) is widely used in NLP. It is not necessary in here, but mentioned to intoduce it to you.

In [6]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

text = 'How to be a grown-up at work: replace "I don\'t want to do that" with "Ok, great!".'
print("before:", text)
print("after:", preprocess(text))

before: How to be a grown-up at work: replace "I don't want to do that" with "Ok, great!".
after: how to be a grown-up at work : replace " i don't want to do that " with " ok , great ! " .


In [9]:
texts_train

array(['... oh and what kind of person WATCHES another persons mytalk page? Stalker!',
       '"\n\nComment\n""But it\'s easier to be lazy and accuse an editor of ownership than to take the time to see what those edits are.""\nBut I suppose reading isn\'t your strong suit, is it?\nPlease see the warning I left for you on your talk page; admins are not above the rules at Wikipedia. STOP the personal attacks.\n I personally feel that the list of all 88 signatories does not detract from the article in any way; properly cited, it simply names people who voluntarily rushed to judgment against these three white boys. It was their decision to get involved in this incident; they should feel proud to be included here. Cheers.  | Talk \np.s. I also have felt (for a long time) that you\'ve attempted to gain \'ownership\' of every article concerning this hoax, as well as any article mentioning Crystal Gail Mangum.  | Talk "',
       'henry lee lucas was my dad im pretty sure i know the information

In [10]:
str(texts_train)

'[\'... oh and what kind of person WATCHES another persons mytalk page? Stalker!\'\n \'"\\n\\nComment\\n""But it\\\'s easier to be lazy and accuse an editor of ownership than to take the time to see what those edits are.""\\nBut I suppose reading isn\\\'t your strong suit, is it?\\nPlease see the warning I left for you on your talk page; admins are not above the rules at Wikipedia. STOP the personal attacks.\\n I personally feel that the list of all 88 signatories does not detract from the article in any way; properly cited, it simply names people who voluntarily rushed to judgment against these three white boys. It was their decision to get involved in this incident; they should feel proud to be included here. Cheers.  | Talk \\np.s. I also have felt (for a long time) that you\\\'ve attempted to gain \\\'ownership\\\' of every article concerning this hoax, as well as any article mentioning Crystal Gail Mangum.  | Talk "\'\n \'henry lee lucas was my dad im pretty sure i know the inform

In [11]:
texts_test

array(['You need a life \n\nREALLY BAD.......................BITCH!',
       'User_talk:TShilo12#Witkacy_WP:POINT  Also, Jayjg got',
       'Your blocks do not deter me \nI may be blocked but you are still an asshole. And coward.  93.138.126.185',
       '"\n\n More Answers \n\n \nPATTY SAID\n\n<<  It clearly shows in the history that others furthered the breed by taking them to America and England. Thus the same with the Shiloh Shepherd. Some facts: While the ISSR was founded in 1991 as the first Shiloh registry, it was only 6 short years later that the first non ISSR registry and club was formed. Thus the ISSR was only the sole Shiloh Registry for 6 years while there have been non ISSR registries for 8.  >>\n\nTINA\'S  REPLY\n\nROTF ... Talk about ""smoke & mirrors"" ... WOW!!  Now let\'s look at some *varifiable* FACTS!\n\n1. The ISSR was formed to CONTINUE the data collection that I started in 1974!!  When it was set up I had documented data on MY breeding stock going all the way b

In [7]:
# task: preprocess each comment in train and test

# data_tok = [tokenizer.tokenize(x.lower()) for x in data]

texts_train = [preprocess(x) for x in texts_train] #<YOUR CODE>
texts_test  = [preprocess(x) for x in texts_test]  #<YOUR CODE>

In [13]:
texts_train

['... oh and what kind of person watches another persons mytalk page ? stalker !',
 '" comment " " but it\'s easier to be lazy and accuse an editor of ownership than to take the time to see what those edits are . " " but i suppose reading isn\'t your strong suit , is it ? please see the warning i left for you on your talk page ; admins are not above the rules at wikipedia . stop the personal attacks . i personally feel that the list of all 88 signatories does not detract from the article in any way ; properly cited , it simply names people who voluntarily rushed to judgment against these three white boys . it was their decision to get involved in this incident ; they should feel proud to be included here . cheers . | talk p . s . i also have felt ( for a long time ) that you\'ve attempted to gain \' ownership \' of every article concerning this hoax , as well as any article mentioning crystal gail mangum . | talk "',
 'henry lee lucas was my dad im pretty sure i know the information be

In [14]:
texts_train[5]

'who cares anymore . they attack with impunity .'

In [8]:
# Small check that everything is done properly
assert texts_train[5] ==  'who cares anymore . they attack with impunity .'
assert texts_test[89] == 'hey todds ! quick q ? why are you so gay'
assert len(texts_test) == len(y_test)

### Step 1: bag of words

One traditional approach to such problem is to use bag of words features:
1. build a vocabulary of frequent words (use train data only)
2. for each training sample, count the number of times a word occurs in it (for each word in vocabulary).
3. consider this count a feature for some classifier

__Note:__ in practice, you can compute such features using sklearn. __Please don't do that in the current assignment, though.__
* `from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer`

In [9]:
import collections
collections.Counter(texts_train)

Counter({'... oh and what kind of person watches another persons mytalk page ? stalker !': 1,
         '" comment " " but it\'s easier to be lazy and accuse an editor of ownership than to take the time to see what those edits are . " " but i suppose reading isn\'t your strong suit , is it ? please see the warning i left for you on your talk page ; admins are not above the rules at wikipedia . stop the personal attacks . i personally feel that the list of all 88 signatories does not detract from the article in any way ; properly cited , it simply names people who voluntarily rushed to judgment against these three white boys . it was their decision to get involved in this incident ; they should feel proud to be included here . cheers . | talk p . s . i also have felt ( for a long time ) that you\'ve attempted to gain \' ownership \' of every article concerning this hoax , as well as any article mentioning crystal gail mangum . | talk "': 1,
         'henry lee lucas was my dad im pretty 

In [17]:
from features import BoW

# task: find up to k most frequent tokens in texts_train,
# sort them by number of occurences (highest first)
k = min(10000, len(set(' '.join(texts_train).split())))
print(k)

#<YOUR CODE>

bow = BoW(k)
bow.fit(texts_train, y_train)
#print('b')
#print('example features:', sorted(bow.get_vocabulary())[::100])

5707
aaaaaaaaaaaaa
b


In [18]:
X_train_bow = bow.transform(texts_train)
X_test_bow = bow.transform(texts_test)

AssertionError: 

In [ ]:
# Small check that everything is done properly
bow_vocabulary = bow.get_vocabulary()
k_max = len(set(' '.join(texts_train).split()))
assert X_train_bow.shape == (len(texts_train), min(k, k_max))
assert X_test_bow.shape == (len(texts_test), min(k, k_max))
assert np.all(X_train_bow[5:10].sum(-1) == np.array([len(s.split()) for s in  texts_train[5:10]]))
assert len(bow_vocabulary) <= min(k, k_max)
assert X_train_bow[6, bow_vocabulary.index('.')] == texts_train[6].split().count('.')

Now let's do the trick with `sklearn` logistic regression implementation:

In [ ]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression().fit(X_train_bow, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

for name, X, y, model in [
    ('train', X_train_bow, y_train, bow_model),
    ('test ', X_test_bow, y_test, bow_model)
]:
    proba = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, proba)
    plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (name, auc))

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.legend(fontsize='large')
plt.grid()

Seems alright. Now let's create the simple logistic regression using PyTorch. Just like in the classwork.

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

from sklearn.metrics import accuracy_score

In [ ]:
from utils import plot_train_process

In [ ]:
model = nn.Sequential()

model.add_module('l1', ### YOUR CODE HERE
### YOUR CODE HERE

Remember what we discussed about loss functions! `nn.CrossEntropyLoss` combines both log-softmax and `NLLLoss`.

__Be careful with it! Criterion `nn.CrossEntropyLoss` with still work with log-softmax output, but it won't allow you to converge to the optimum.__ Next comes small demonstration:

In [ ]:
# loss_function = nn.NLLLoss()
loss_function = nn.CrossEntropyLoss()

In [ ]:
opt = ### YOUR CODE HERE

In [ ]:
X_train_bow_torch = ### YOUR CODE HERE
X_test_bow_torch = ### YOUR CODE HERE

y_train_torch = ### YOUR CODE HERE
y_test_torch = ### YOUR CODE HERE

Let's test that everything is fine

In [ ]:
# example loss
loss = loss_function(model(X_train_bow_torch[:3]), y_train_torch[:3])

In [ ]:
assert type(loss.item()) == float

Here comes small function to train the model. In future we will take in into separate file, but for this homework it's ok to implement it here. 

In [ ]:
def train_model(
    model,
    opt,
    lr_scheduler,
    X_train_torch,
    y_train_torch,
    X_val_torch,
    y_val_torch,
    n_iterations=500,
    batch_size=32,
    warm_start=False,
    show_plots=True,
    eval_every=10
):
    if not warm_start:
        for name, module in model.named_children():
            print('resetting ', name)
            try:
                module.reset_parameters()
            except AttributeError as e:
                print('Cannot reset {} module parameters: {}'.format(name, e))

    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []

    local_train_loss_history = []
    local_train_acc_history = []
    for i in range(n_iterations):

        # sample 256 random observations
        ix = np.random.randint(0, len(X_train_torch), batch_size)
        x_batch = X_train_torch[ix]
        y_batch = y_train_torch[ix]

        # predict log-probabilities or logits
        y_predicted = ### YOUR CODE

        # compute loss, just like before
        ### YOUR CODE


        # compute gradients
        ### YOUR CODE

        # Adam step
        ### YOUR CODE

        # clear gradients
        ### YOUR CODE


        local_train_loss_history.append(loss.data.numpy())
        local_train_acc_history.append(
            accuracy_score(
                y_batch.to('cpu').detach().numpy(),
                y_predicted.to('cpu').detach().numpy().argmax(axis=1)
            )
        )

        if i % eval_every == 0:
            train_loss_history.append(np.mean(local_train_loss_history))
            train_acc_history.append(np.mean(local_train_acc_history))
            local_train_loss_history, local_train_acc_history = [], []

            predictions_val = model(X_val_torch)
            val_loss_history.append(loss_function(predictions_val, y_val_torch).to('cpu').detach().item())

            acc_score_val = accuracy_score(y_val_torch.cpu().numpy(), predictions_val.to('cpu').detach().numpy().argmax(axis=1))
            val_acc_history.append(acc_score_val)
            lr_scheduler.step(train_loss_history[-1])

            if show_plots:
                display.clear_output(wait=True)
                plot_train_process(train_loss_history, val_loss_history, train_acc_history, val_acc_history)
    return model

Let's run it on the data. Note, that here we use the `test` part of the data for validation. It's not so good idea in general, but in this task our main goal is practice.

In [ ]:
train_model(model, opt, lr_scheduler, X_train_bow_torch, y_train_torch, X_test_bow_torch, y_test_torch)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

for name, X, y, model in [
    ('train', X_train_bow_torch, y_train, model),
    ('test ', X_test_bow_torch, y_test, model)
]:
    proba = model(X).detach().cpu().numpy()[:, 1]
    auc = roc_auc_score(y, proba)
    plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (name, auc))

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.legend(fontsize='large')
plt.grid()

Try to vary the number of tokens `k` and check how the model performance changes. Show it on a plot.

In [ ]:
# Your beautiful code here

### Step 2: implement TF-IDF features

Not all words are equally useful. One can prioritize rare words and downscale words like "and"/"or" by using __tf-idf features__. This abbreviation stands for __text frequency/inverse document frequence__ and means exactly that:

$$ feature_i = { Count(word_i \in x) \times { log {N \over Count(word_i \in D) + \alpha} }}, $$


where x is a single text, D is your dataset (a collection of texts), N is a total number of documents and $\alpha$ is a smoothing hyperparameter (typically 1). 
And $Count(word_i \in D)$ is the number of documents where $word_i$ appears.

It may also be a good idea to normalize each data sample after computing tf-idf features.

__Your task:__ implement tf-idf features, train a model and evaluate ROC curve. Compare it with basic BagOfWords model from above.

__Please don't use sklearn/nltk builtin tf-idf vectorizers in your solution :)__ You can still use 'em for debugging though.

Blog post about implementing the TF-IDF features from scratch: https://triton.ml/blog/tf-idf-from-scratch

In [ ]:
from features import TfIdf

# Your beautiful code here

Same stuff about moel and optimizers here (or just omit it, if you are using the same model as before).

In [ ]:
### YOUR CODE HERE

In [ ]:
X_train_tfidf_torch = ### YOUR CODE HERE
X_test_tfidf_torch = ### YOUR CODE HERE

y_train_torch = ### YOUR CODE HERE
y_test_torch = ### YOUR CODE HERE

Fit your model to the data. No not hesitate to vary number of iterations, learning rate and so on.

_Note: due to very small dataset, increasing the complexity of the network might not be the best idea._

### Step 3: Comparing it with Naive Bayes

Naive Bayes classifier is a good choice for such small problems. Try to tune it for both BOW and TF-iDF features. Compare the results with Logistic Regression.

In [ ]:
# Your beautiful code here

Shape some thoughts on the results you aquired. Which model has show the best performance? Did changing the learning rate/lr scheduler help?

_Your beautiful thoughts here_

### Step 4: Using the external knowledge.

Use the `gensim` word2vec pretrained model to translate words into vectors. Use several models with this new encoding technique. Compare the results, share your thoughts.

In [ ]:
# Your beautiful code here